In [1]:
### part of the preprocessing step for training, used for testing adding 12 month target
### test loimport numpy as np
import xarray as xr
from fsspec.implementations.http import HTTPFileSystem
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xarray as xr
import math
import sys
import time
import geopandas as gpd
from pyproj import Transformer
import pickle as pkl
import json 
import numpy as np
import rasterio

from rasterio.features import rasterize
import affine


from datetime import datetime
from functools import wraps
from fsspec.implementations.http import HTTPFileSystem
# sys.path.append('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/custom_functions')

sys.path.append('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/Tempo-SpaceTech/veg_forecasting/util/')
# sys.path.append('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/Tempo-SpaceTech/veg_forecasting/util/dea_tools')

# '/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/Tempo-SpaceTech/veg_forecasting/util/'
from dea_functions import request_DEA_S2_data
from dea_functions import request_DEA_fc_data

import space_blob
from space_blob.user_blob import create_blob_stream
from space_blob.user_blob import write_blob_client
from space_blob.user_blob import get_blobnames

... source package location.../mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/space_packs/space_blob/space_blob/user_blob
Successfully reading blob authentication ...


In [14]:
def mean_value_one_var(ds, var, min_val=-1.5, max_val=1.5):
#     return np.nanmean(ds[var].data.flatten())
    """ Purpose: obtain the mean value of a band 'var' from an xarray.Dataset, exlcuding outliers and nan/inf values
        Parameters: 
        --------
        ds: xarray.Dataset, 
        var: string,
            a band contained in ds
        min_val: any numerical type,
            the minimum acceptable value for var
        max_val: any numerical type,
            the maximum acceptable value for var
        
        I have set min_val and max_val to -1.5 and 1.5 as default because so far I only use this function for EVI (CC)
    """
    ### TT-todo: the code below can be faster by making use of list comp. or np.array
    evi_vals = []
    time_index = 0
    for time in ds.time.data:
        evi_val = ds.isel(time=time_index)['evi'].data[0][0]
        if evi_val <= max_val and evi_val >= min_val:
            evi_vals.append(evi_val)
        time_index += 1

    return np.nanmean(np.array(evi_vals))

def mean_value_dvar(ds, var='evi', min_val=-1.5, max_val=1.5):
#     return np.nanmean(ds[var].data.flatten())
    """ Purpose: obtain the mean value of a band 'var' from an xarray.Dataset, exlcuding outliers and nan/inf values
        Parameters: 
        --------
        ds: xarray.Dataset, 
        var: string,
            a band contained in ds
        min_val: any numerical type,
            the minimum acceptable value for var
        max_val: any numerical type,
            the maximum acceptable value for var
        
        I have set min_val and max_val to -1.5 and 1.5 as default because so far I only use this function for EVI (CC)
    """
    ### TT-todo: the code below can be faster by making use of list comp. or np.array
    evi_vals = []
    time_index = 0
    for time in ds.time.data:
        evi_val = ds.isel(time=time_index)['evi'].data
        if evi_val <= max_val and evi_val >= min_val:
            evi_vals.append(evi_val)
        time_index += 1

    return np.nanmean(np.array(evi_vals))


def get_previous_month_string(month_year):
    """ Purpose: return a string representing the previous month
        Parameters: 
        --------
        request: string,
                 expected format YYYY-MM
        e.g. 2017-01 will return 2016-12
    """
    for p in pd.period_range(month_year, month_year, freq='M')-1:
        return str(p)
    
def find_time_index(request, ds):
    """ Purpose: find a time index in the dataset ds which match the request
        Parameters: 
        --------
        request: string,
                 expected format YYYY-MM-DD or YYYY-MM
                 YYYY-MM will cause the function to return the first time index from the requested month
        ds: xarray.Dataset with a time dimension
        ### TT-todo: the code below can be faster by making use of Dataset.where() 
    """
    if len(request) == 7 or len(request) == 10:
        if type(ds['time'].data[0]) == np.datetime64:
            for i, s in enumerate(list(np.datetime_as_string(ds['time'].data))):
                if request in s:
                    return i
        if type(ds['time'].data[0]) == str:
            for i, s in enumerate(ds['time'].data):
                if request in s:
                    return i
            
def find_all_month_indices(request, ds):
    ### TT-todo: the code below can be faster by making use of Dataset.where() and/or list comp.
    if len(request)==7:
        indices = []
        for i, s in enumerate(list(np.datetime_as_string(ds['time'].data))):
            if request in s:
                indices.append(i)
        return indices
    
def find_all_year_indices(request, ds):
    ### TT-todo: the code below can be faster by making use of Dataset.where() and/or list comp.
    if len(request)==4:
        indices = []
        for i, s in enumerate(list(np.datetime_as_string(ds['time'].data))):
            if request in s:
                indices.append(i)
        return indices

def bilinear_interpolation(x, y, points):
    '''Interpolate (x,y) from values associated with four points.

    The four points are a list of four triplets:  (x, y, value).
    The four points can be in any order.  They should form a rectangle.

        >>> bilinear_interpolation(12, 5.5,
        ...                        [(10, 4, 100),
        ...                         (20, 4, 200),
        ...                         (10, 6, 150),
        ...                         (20, 6, 300)])
        165.0

    '''
    # Got this from https://stackoverflow.com/questions/8661537/how-to-perform-bilinear-interpolation-in-python :D

    points = sorted(points)               # order points by x, then by y
    (x1, y1, q11), (_x1, y2, q12), (x2, _y1, q21), (_x2, _y2, q22) = points

    return (q11 * (x2 - x) * (y2 - y) +
            q21 * (x - x1) * (y2 - y) +
            q12 * (x2 - x) * (y - y1) +
            q22 * (x - x1) * (y - y1)
           ) / ((x2 - x1) * (y2 - y1) + 0.0)

def interpolate_single_band(ds_env, lat, lon, band, date, crs="EPSG:3577"):
    """ Purpose: interpolate a single value for a single band at the specified latitude and longitude
        Parameters: 
        --------
        ds_env: xarray.Dataset
        lat: float, representing latitude in the specified crs
        lon: float, representing longitude in the specified crs
        band: string, representing a band in the ds_env dataset
        date: string, in the format YYYY-MM 
        crs: string, representing the crs of the lat and lon values
    """
    
    transformer = Transformer.from_crs(crs, "EPSG:4326")
    s2_lat, s2_lon = transformer.transform(lon, lat)
    
    time_index = find_time_index(date, ds_env)

    min_lat = math.floor(s2_lat*10)/10
    max_lat = math.ceil(s2_lat*10)/10
    min_lon = math.floor(s2_lon*10)/10
    max_lon = math.ceil(s2_lon*10)/10

    filt1 = ds_env['latitude'] == min_lat
    filt2 = ds_env['latitude'] == max_lat
    filt3 = ds_env['longitude'] == min_lon
    filt4 = ds_env['longitude'] == max_lon

    min_lat_index = list(filt1.data).index(True)
    max_lat_index = list(filt2.data).index(True)
    min_lon_index = list(filt3.data).index(True)
    max_lon_index = list(filt4.data).index(True)

    points = ((min_lat, min_lon, ds_env.isel(time=time_index)[band].data[min_lat_index][min_lon_index]),
              (max_lat, min_lon, ds_env.isel(time=time_index)[band].data[max_lat_index][min_lon_index]),
              (min_lat, max_lon, ds_env.isel(time=time_index)[band].data[min_lat_index][max_lon_index]),
              (max_lat, max_lon, ds_env.isel(time=time_index)[band].data[max_lat_index][max_lon_index]))

    return bilinear_interpolation(s2_lat, s2_lon, points)


def locate_index(ds_fc, lat_pv, lon_pv, itime=28, var_s='PV_PC_90'):
    """
    Purpose: return the index for a given value set in lat and lon
    Input para: 
             ds_fc: xarray dataset
             lat_pv, lon_pv: coordinates, floats in EPSG:4326
     Optional para:  itime=28, time index, 
                        var_s='PV_PC_90', a variable name for getting Xarray Dataset
    Return:  ix, iy, itime = index in x, y, time, 
             x, y, time, =  matched values
    """
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3577")
    # Note the order of the output
    s_lon_3577, s_lat_3577 = transformer.transform(lat_pv, lon_pv)  ## .transformat(lat/y, lon/x)
    # value for the time index
    time_pv = ds_fc.coords['time'][itime].data
    ds_pv_sel = ds_fc[var_s].sel(x=s_lon_3577, y=s_lat_3577, time=time_pv, method='nearest')
    filt1 =  ds_fc[var_s].x == ds_pv_sel.x.data
    ix = list(filt1.data).index(True)
    filt2 =  ds_fc[var_s].y == ds_pv_sel.y.data
    iy = list(filt2.data).index(True)
    return ix, iy, itime, ds_pv_sel.x.data, ds_pv_sel.y.data, time_pv


def get_mask_sh(ds, gdf):
    crs = ds.crs
    # transform = ds.transform
    # transform=affine.Affine(10.0, 0.0, ds.coords['x'].data.min(),
    #                         0.0, -10.0, ds.coords['y'].data.max())
    transform=affine.Affine(10.0, 0.0, ds.coords['x'].data.min()-5,
                        0.0, -10.0, ds.coords['y'].data.max()+5)
    y, x = len(ds.coords['y']), len(ds.coords['x'])
    try:
        gdf_reproj = gdf.to_crs(crs=crs)
    except:
        # Sometimes the crs can be a datacube utils CRS object
        # so convert to string before reprojecting
        gdf_reproj = gdf.to_crs(crs=str(crs))
    # Use geometry directly (will produce a boolean numpy array)
    shapes = gdf_reproj.geometry
    # Rasterise shapes into an array
    arr = rasterize(shapes=shapes,
                    out_shape=(y, x),
                    transform=transform)
    # Convert result to a xarray.DataArray
    xarr = xr.DataArray(arr,
                        coords=(ds.coords['y'], ds.coords['x']),
                        dims=('y', 'x'),
                        attrs=ds.attrs)
    return xarr

In [3]:
### e.g., Masks for the railway
###  other formats require modification in #Mask cell below
home_dir = '/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/'
mask_file = home_dir + "data_tmp/W_angelas_all7poly.geojson"

loc_name = 'W_angelas'
veg_type = 'npv'
# fc_year_range = ("2015", "2020")
# out_fc = f'{loc_name}_fc_annual_cover_15_20_test.pkl'        ### annual fractional cover dataset
# out_fc = 'odc_vm_data/odc_outputs/W_angelas_fc_annual_cover_15_20_test.pkl'
file_pv_coords = f'{loc_name}_all_{veg_type}_hc_coords_test.csv'    ### output coordinates for n/PV pixels
train_file_csv = f'{loc_name}_{veg_type}_100px_monthly_train_test.csv'   ### output training dataframe name in csv
train_file_pkl = f'{loc_name}_{veg_type}_100px_monthly_train_test.pkl'   ### output training dataframe name in pkl

In [4]:
fn = 'odc_vm_data/odc_outputs/co2_mm_gl_url.txt'
fn_url = create_blob_stream(fn, t_expire=8)
fs = HTTPFileSystem()
### To-Do: find a better way to write this so that the line number like 57 is auto-detected?
column_names = fs.open(fn_url).readlines()[57][2:-1].decode().split()
co2_df = pd.DataFrame(columns=column_names)

with fs.open(fn_url) as f:
    for line in f.readlines()[58:]:
        vals = line[2:-1].decode().split()
        dict_vals = {}
        for i in range(len(column_names)):
            dict_vals[column_names[i]] = vals[i]
        co2_df = co2_df.append(dict_vals, ignore_index=True) 
    
co2_df['month-year'] = co2_df['year'].astype('str') + '-' + co2_df['month'].astype('str').str.rjust(2,'0')
co2_df = co2_df.set_index(['month-year'])
co2_df.drop(columns=['year', 'month', 'decimal', 'trend'], inplace=True)
co2_df['average'] = co2_df['average'].astype('float')

#  Create a url stream for the monthly climate data
fn = '/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc'
fn_url = create_blob_stream(fn, t_expire=8)
fs = HTTPFileSystem()
ds_env = xr.open_dataset(fs.open(fn_url))
ds_env = ds_env.drop(['lai_hv'])   # cuz lai_hv column is problematic, mostly zeros

In [5]:
checkn = get_blobnames('')

/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/mim_data_optical_water_stdev.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/mim_models_data.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/rain_simple.csv
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ty-test-oct8/code/Users/Tiantian.Yuan/data_tmp/rain_simple_max.csv
HC_pred_green_from2021May_v1.tif
HC_pv_100px_monthly_train.csv
Mesa_pv_100px_monthly_train.csv
data/32a801906097225af35a5848adb124ba/response.tiff
gpr_updated.csv
hello.txt
master-auto_geometry.csv
mim_models_data.csv
model_features.csv
odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2018-12_test.pkl
odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2021-10_test.pkl
odc_vm_data/odc_outputs/W_angelas_2_evibands_2019-01_2021-10_test.pkl
o

In [6]:
### We only need to do this one time
# query_s2_all = True         # set to true if loading data directly from DEA
# write_to_local = False    # set to true if writing the pkl file to the current folder
write_to_blob = False    # set to true if writing the pkl file to a blob location

### coodinate bounds, must be in CRS of 4326 
max_lon_4326 = 118.760256
min_lon_4326 = 118.73415
max_lat_4326 = -22.940174
min_lat_4326 = -22.979836

lon_range = (min_lon_4326, max_lon_4326)
lat_range = (min_lat_4326, max_lat_4326)
bbox = [min_lon_4326, min_lat_4326, max_lon_4326, max_lat_4326]

# s2_query_dates = ('2015-12', '2021-10')
# ds2_all = None
# s2_query_dates_all = [('2019-01', '2021-10')]
# # s2_query_dates_all = [('2015-12', '2018-12'), ('2019-01', '2021-10')]

# # for s2_query_dates in s2_query_dates_all:
# # s2_query_dates = ('2015-12', '2018-12')  # takes 16mins
# s2_query_dates =  ('2019-01', '2021-10')
# s2_out_file = f'{loc_name}_2_evibands_{s2_query_dates[0]}_{s2_query_dates[1]}_test.pkl'    ## pickle file to be saved
# ### query all the S2 data in one go
# # date_range, bbox, bands=None, resolution=10, min_gooddata=0.0, resampling=rasterio.enums.Resampling.nearest, verbose=False
# bands = ['nbart_red', 'nbart_blue', 'nbart_nir_1']
# ds2_all = request_DEA_S2_data(s2_query_dates, bbox,
#                             bands=bands, resolution=10, min_gooddata=0.8, verbose=True)
# ds2_all['evi'] = 2.5*((ds2_all.nbart_nir_1 - ds2_all.nbart_red) / (ds2_all.nbart_nir_1 + 6*ds2_all.nbart_red - 7.5*ds2_all.nbart_blue +1))
# if write_to_blob:
#     pkl_ds2 = pkl.dumps(ds2_all)
#     blob_client = write_blob_client(s2_out_file, parent_dir='odc_vm_data/odc_outputs/')
#     blob_client.upload_blob(pkl_ds2, overwrite=True)

In [9]:
# del ds2_all

In [7]:
# concat two pkl files representing 2 time periods of S2 data
# fblob_s2_y1 = 'odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2018-12_test.pkl'
# fblob_s2_y2 = 'odc_vm_data/odc_outputs/W_angelas_2_evibands_2019-01_2021-10_test.pkl'
# fx_url = create_blob_stream(fblob_s2_y1, t_expire=1)
# fs_x = HTTPFileSystem()
# with fs_x.open(fx_url, 'rb') as f:
#     ds2_part1 = pkl.load(f)

# fx_url2 = create_blob_stream(fblob_s2_y2, t_expire=1)
# fs_x2 = HTTPFileSystem()
# with fs_x2.open(fx_url2, 'rb') as f:
#     ds2_part2 = pkl.load(f)
# ds2_all = xr.concat([ds2_part1, ds2_part2], dim='time')

In [7]:
fblob_s2_y1 = 'odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2021-10_test.pkl'
fx_url = create_blob_stream(fblob_s2_y1, t_expire=1)
fs_x = HTTPFileSystem()
with fs_x.open(fx_url, 'rb') as f:
    ds2_all = pkl.load(f)

In [16]:
### test request_DEA_fc_data(year_range, bbox, bands=None, resolution=25, resampling=rasterio.enums.Resampling.nearest, verbose=False)
### query all the S2 data in one go
# write_to_blob = True    # set to true if writing the pkl file to a blob location

# fc_year_range = ("2015", "2020")

# ds_fc = request_DEA_fc_data(fc_year_range, bbox,
#                             resolution=10, verbose=True)
# if write_to_blob:
#     pkl_fc = pkl.dumps(ds_fc)
#     blob_client = write_blob_client(out_fc, parent_dir='odc_vm_data/odc_outputs/')
#     blob_client.upload_blob(pkl_fc, overwrite=True)

2015-01-01 2020-12-31
loading timestamp:  2015-01-01T00:00:00Z
loading timestamp:  2016-01-01T00:00:00Z
loading timestamp:  2017-01-01T00:00:00Z
loading timestamp:  2018-01-01T00:00:00Z
loading timestamp:  2019-01-01T00:00:00Z
loading timestamp:  2020-01-01T00:00:00Z
6 timestamps loaded


In [9]:
fblob_fc = 'odc_vm_data/odc_outputs/W_angelas_fc_annual_cover_15_20_test.pkl'
fx_url_fc = create_blob_stream(fblob_fc, t_expire=1)
# fs_x_fc = HTTPFileSystem()
with HTTPFileSystem().open(fx_url_fc, 'rb') as f:
    ds_fc = pkl.load(f)

In [10]:
ds_fc

<xarray.Dataset>
Dimensions:      (time: 6, x: 310, y: 469)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01 2016-01-01 ... 2020-01-01
  * y            (y) float64 -2.542e+06 -2.542e+06 ... -2.546e+06 -2.546e+06
  * x            (x) float64 -1.345e+06 -1.345e+06 ... -1.342e+06 -1.342e+06
    spatial_ref  int32 3577
Data variables:
    BS_PC_10     (time, y, x) int16 43 47 47 48 48 48 43 ... 52 52 53 53 60 60
    PV_PC_10     (time, y, x) int16 0 0 0 0 0 0 0 0 0 0 ... 0 1 1 1 1 1 1 1 0 0
    NPV_PC_10    (time, y, x) int16 11 12 12 13 13 13 13 ... 32 32 30 30 22 22
    BS_PC_50     (time, y, x) int16 47 52 52 52 52 52 49 ... 59 59 60 60 74 74
    PV_PC_50     (time, y, x) int16 8 4 4 3 3 3 7 7 10 10 ... 4 4 4 4 4 4 4 0 0
    NPV_PC_50    (time, y, x) int16 43 43 43 42 42 42 42 ... 34 34 33 33 24 24
    BS_PC_90     (time, y, x) int16 88 86 86 86 86 86 86 ... 63 63 66 66 77 77
    PV_PC_90     (time, y, x) int16 12 8 8 5 5 5 10 10 ... 11 11 11 11 11 11 4 4
    NPV_PC_90    (time, y, x) int16 45 44 44 46 46 46 48 ... 37 37 37 37 31 31
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Next: Check if we can load xr_rasterzie 

In [11]:
ds2_all

<xarray.Dataset>
Dimensions:      (time: 235, x: 310, y: 469)
Coordinates:
  * time         (time) datetime64[ns] 2015-12-04T02:20:12.098000 ... 2021-07...
  * y            (y) float64 -2.542e+06 -2.542e+06 ... -2.546e+06 -2.546e+06
  * x            (x) float64 -1.345e+06 -1.345e+06 ... -1.342e+06 -1.342e+06
    spatial_ref  int32 3577
Data variables:
    nbart_red    (time, y, x) float32 1.115e+03 1.022e+03 ... 1.1e+03 1.125e+03
    nbart_blue   (time, y, x) float32 394.0 379.0 405.0 ... 317.0 288.0 310.0
    nbart_nir_1  (time, y, x) float32 1.336e+03 1.245e+03 ... 1.55e+03 1.603e+03
    evi          (time, y, x) float32 0.1089 0.1229 0.1102 ... 0.1878 0.1982
Attributes:
    crs:           epsg:3577
    grid_mapping:  spatial_ref

In [12]:
ds_fc.attrs['crs'] = "epsg:3577"
print(ds_fc.crs)

epsg:3577


In [15]:
### Mask cell
# Mask the pixels to only include the area near the railway line
gdf = gpd.read_file(mask_file)
#  Convert mask file crs to 3577
gdf_3577 = gdf.to_crs("EPSG:3577")
mask = get_mask_sh(ds2_all, gdf_3577)
ds_fc_m = ds_fc.where(mask)
ds_s2_m = ds2_all.where(mask)
# Create a Dataset from the data arrays, which has the same dimensions as ds from Sentinel-2

In [ ]:
### now select pixel types
create_new = True     ### if true, write out the dataframe in csv file
random_select = 0     ### randomly select 100 pixels,  if for the whole region, then = 0
write_to_blob = True
PV = False
NPV = True

masked_use = True
if create_new:
    ds_fc_time_median = ds_fc.median(dim='time', skipna=True)
    if PV:
        filt1 = ds_fc_time_median['PV_PC_90'] > ds_fc_time_median['NPV_PC_90']
        filt2 = ds_fc_time_median['PV_PC_90'] > ds_fc_time_median['BS_PC_90']
        ###  condtion for training
        cond_pv = (ds_fc_time_median['PV_PC_90'] > 20) & (ds_fc_time_median['BS_PC_90'] < 60)
#         cond_pv = (ds_fc_time_median['PV_PC_90'] > 20) & (ds_fc_time_median['BS_PC_90'] < 60)
#         cond_pv = (filt1 & filt2)     # this results in few data
    elif NPV:
        filt1 = (ds_fc_time_median['NPV_PC_90'] > 45) & (ds_fc_time_median['PV_PC_90'] < 20)
        filt2 =  ds_fc_time_median['BS_PC_90'] < 60
        ###  condtion for training
        cond_pv = (filt1 & filt2) 
    else:
        cond_pv = ds_fc_time_median['BS_PC_90'] >= 60



    if masked_use:
        mds_fc_time_median = ds_fc_m.median(dim='time', skipna=True)   ## for masked regions
        mcony = mds_fc_time_median['PV_PC_90'].where(cond_pv, drop=True).squeeze()
    else:        
        mcony = ds_fc_time_median['PV_PC_90'].where(cond_pv, drop=True).squeeze()

    mcony_df = mcony.to_dataframe().dropna()
    # mcony_df.info()

    mcony_df.reset_index(inplace=True)
    mcony_df.drop_duplicates(subset=['y','x'], keep='first', inplace=True)

    # mcony_df.info()
    transformer = Transformer.from_crs("EPSG:3577", "EPSG:4326")
    y_4326, x_4326 = transformer.transform(mcony_df.x.values, mcony_df.y.values)
    mcony_df['y_4326'] = y_4326
    mcony_df['x_4326'] = x_4326

    ### random samples
    if random_select > 0:
        random_pv_df = mcony_df.sample(n=random_select, replace=False)
    else:
        random_pv_df = mcony_df.copy()
    random_pv_df.to_csv(file_pv_coords)   ## write av local csv file for the PV coordinates
else:
    random_pv_df = pd.read_csv(file_pv_coords)
    
    
if write_to_blob:
    blob_client = write_blob_client(file_pv_coords, parent_dir='odc_vm_data/odc_outputs/')
    with open(file_pv_coords, 'rb') as data:
        blob_client.upload_blob(data, overwrite=True)